In [ ]:
#!pip install import-ipynb -q

In [ ]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

In [ ]:
import import_ipynb
import b4_pre_processing_step
import c2_training_xgboost_step

In [ ]:
pre_processing_step = b4_pre_processing_step.create_pre_processing_step()

training_step = c2_training_xgboost_step.create_training_step()
training_step.add_depends_on([pre_processing_step])

from sagemaker.workflow.step_collections import RegisterModel
register_model_step = RegisterModel(
     name="Register_Model",
     estimator= training_step.estimator,
     model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
     content_types=["application/json"],
     response_types=["application/json"],
     #inference_instances=['ml.m5.large'],     
     model_package_group_name='poc-model'
)

In [ ]:
# create pipeline
pipeline = Pipeline(
    name='poc-pipeline',
    steps=[pre_processing_step, training_step, register_model_step]
)

pipeline_definition = pipeline.definition()
print(pipeline_definition)

In [ ]:
# upload to sagemaker
pipeline.upsert(role_arn=sagemaker.get_execution_role())

In [ ]:
# pipeline.start()